In [1]:
#our virtual environment is this: source /Users/dkim1998/Documents/virtual_environment/virtual_environment/bin/activate
import pandas as pd 
import numpy as np 
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import evaluate
import transformers


/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/trans

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

In [1]:
###First step: Lets set up the data we're testing 
#train_dataset = load_dataset('gem/viggo', split='train')
#eval_dataset = load_dataset('gem/viggo', split='validation')
#test_dataset = load_dataset('gem/viggo', split='test')
file_path1 = "/Users/dkim1998/Downloads/LLM_PII_Project_Code/fake_pii_data_unlabeled_training.txt"
with open(file_path1, 'r') as file:
    training_data = file.read()

file_path2 = "/Users/dkim1998/Downloads/fake_pii_data_very_dirty_labeled_validation.txt"
with open(file_path2, 'r') as file:
    validation_data = file.read()

file_path3 = "/Users/dkim1998/Downloads/fake_pii_data_very_dirty_labeled_testing.txt"
with open(file_path3, 'r') as file:
    testing_data = file.read()

In [4]:
print(testing_data)

name: Abigail Anderson
address: Roy Rapids
email: rmcknight@example.
phone: (782)548-0354
ssn: 388-53-4785
driver_license: 867 GBN
passport_number: YLSK37925099
credit_card: JCB 16 digit
Kenneth Dunn
3587466832526667 09/31
CVC: 933

bank_account: KZCF48672615383818
company: Anderson-Coleman
job_title: Fisheries officer
username: martinkimberly
password: zU4@T6VN#1
url: https://bates.info/
date_of_birth: 03/12/2007
medicine_name: Change car.
disease_name: Back recently.
latitude_longitude: -67.902600, -131.503097
random_text: Side care see look. Wall arm shoulder.
isbn: 978-1-76172-021-5

name: Watkins
address: 57839 Gibson Summit Apt. 866
East Davidmouth, MO 32968
email: calderonpatricia@
phone: 
ssn: 555-11-3644
driver_license: 320-DVE
passport_number: VUKU29656871
credit_card: VISA 16 digit
Adam Ortega
4180854908076908 08/24
CVC: 598

bank_account: BZAV52388906124818
company: Collins-Anderson
job_title: HEALTH PHYSICIST
username: gchapman
password: 3CjYpU8n!F
url: http://www.johnson.

In [3]:
#loads up the model
base_model_id = "/Users/dkim1998/Documents/Mistral_7b_v2"
model = AutoModelForCausalLM.from_pretrained(base_model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [6]:
###Setting up a prompt
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""
Identify the PII data type for each of the following:

1. name: John Doe
2. address: 123 Main St, Anytown, USA
3. email: johndoe@example.com
4. phone: 555-1234
5. ssn: 123-45-6789
6. driver_license: D1234567
7. passport_number: 987654321
8. credit_card: 4111 1111 1111 1111
9. bank_account: 000123456789
10. company: Best Buy
11. job_title: Software Engineer
12. username: johndoe123
13. password: P@ssw0rd!
14. url: https://www.example.com
15. date_of_birth: 01/01/1980
16. medicine_name: Aspirin
17. disease_name: Diabetes
18. latitude_longitude: 40.7128° N, 74.0060° W
19. random_text: Hello world!
20. isbn: 978-3-16-148410-0

For each item, determine its PII data type (e.g., "Best Buy" should be labeled as 'company')."""

    return tokenize(full_prompt)

In [7]:
####Lets keep this seperate and for the sake of simplicity make this into a dataset
from datasets import Dataset, load_dataset
import pandas as pd

def read_text_file_to_dataset(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    data = {'text': lines}
    return Dataset.from_pandas(pd.DataFrame(data))

In [8]:
# Load your text files
training_data = read_text_file_to_dataset("/Users/dkim1998/Downloads/LLM_PII_Project_Code/fake_pii_data_unlabeled_training.txt")
validation_data = read_text_file_to_dataset("/Users/dkim1998/Downloads/fake_pii_data_very_dirty_labeled_validation.txt")


In [9]:
print(training_data)

Dataset({
    features: ['text'],
    num_rows: 1210
})


In [10]:
tokenized_train_dataset = training_data.map(generate_and_tokenize_prompt)
tokenized_val_dataset = validation_data.map(generate_and_tokenize_prompt) #this would combine the prompt to the text data itself

Map: 100%|██████████| 7621/7621 [00:02<00:00, 3153.11 examples/s]


In [11]:
eval_prompt = f"""
Given a target sentence, identify the PII data type present in it. For example, if a sentence mentions 'Best Buy', it should be labeled as 'company'.

### Target Sentence:
{"name"}, {"address"}, {"email"}, {"phone"}, {"ssn"}, {"driver_license"}, {"passport_number"}, {"credit_card"}, {"bank_account"}, {"company"}, {"job_title"}, {"username"}, {"password"}, {"url"}, {"date_of_birth"}, {"medicine_name"}, {"disease_name"}, {"latitude_longitude"}, {"random_text"}, {"isbn"}

### PII Identification:
"""


In [12]:
model_input = tokenizer(eval_prompt, return_tensors="pt")
model.eval()
with torch.no_grad(): #something to note: It said right-padding was detected, in the second run I should change that
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Given a target sentence, identify the PII data type present in it. For example, if a sentence mentions 'Best Buy', it should be labeled as 'company'.

### Target Sentence:
name, address, email, phone, ssn, driver_license, passport_number, credit_card, bank_account, company, job_title, username, password, url, date_of_birth, medicine_name, disease_name, latitude_longitude, random_text, isbn

### PII Identification:

1. name: John Doe, Jane Smith, etc.
2. address: 123 Main St, 456 Elm St, etc.
3. email: john.doe@example.com, jane.smith@example.com, etc.
4. phone: 555-555-5555, 555-555-1234, etc.
5. ssn: 123-45-6789, 987-65-4321, etc.
6. driver_license: ABC-1234, XYZ-5678, etc.
7. passport_number: X1234567, JK987654, etc.
8. credit_card: 1234-5678-9012-3456, 2345-6789-0123-4567, etc.
9. bank_account: 1234567890, 9876543210, etc.
10. company: Best Buy, Apple, Microsoft, etc.
11. job_title: CEO, CTO, Engineer, etc.
12. username: johndoe, janesmith, etc.
13. password: 123456, qwerty, etc.
1

In [13]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [14]:
##Finally we do LoRA
config = LoraConfig(
    r=4,
    lora_alpha=8,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
       # "gate_proj",  #the number of target modules affect the complexity of the module, the more there is the more complex this is
       # "up_proj", #the four in # are great for generating , but not for attention mechanism
       # "down_proj",
       # "lm_head" #if i want to achieve the result of a full finetuning, use them all  3https://www.reddit.com/r/LocalLLaMA/comments/15sgg4m/what_modules_should_i_target_when_training_using/
    ],
    bias="none",
    lora_dropout=0.05,         # Conventional
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

#Trainable Percentage: 0.2927207805812288% indicates that a very small fraction of the total parameters is actually being trained, which aligns with the concept of LoRA (Low-Rank Adaptation) where only a subset of the model's parameters is fine-tuned.
#trainable%: 0.04703666202518836, this occurs if we only use         "q_proj","k_proj","v_proj", "o_proj",

##Extra info on the r and lora_alpha
#r: This parameter represents the rank of the low-rank matrices used in LoRA. A higher rank can potentially capture more complex adaptations but also increases the number of parameters and computational complexity. r=4 means you are using a rank of 4 for these matrices.
#lora_alpha: This is a scaling factor applied to the low-rank matrices. It controls the extent to which the LoRA adjustments impact the original model weights. A higher lora_alpha value means a stronger effect of the LoRA modifications on the model's behavior.

##Extra info on bias, lora_dropout, and task_type
#bias: This setting determines whether to apply biases in the LoRA layers. Setting bias="none" means that no bias terms are added to the LoRA-adapted layers. Including bias can sometimes improve the model's capacity to fit the data, but it also adds more parameters.
#lora_dropout: This is the dropout rate applied to the LoRA layers, set here at 0.05 or 5%. Dropout is a regularization technique used to prevent overfitting. It randomly sets a fraction of input units to 0 during training, which helps in making the model more robust.
#task_type: The task_type="CAUSAL_LM" specifies the type of task for which the model is being adapted. "CAUSAL_LM" indicates a causal language modeling task, typically used in models that generate text based on the preceding context.

##extra info on the target_modules
#"q_proj", "k_proj", "v_proj", and so on, corresponds to a part of the attention mechanism in the Transformer layers.

#"q_proj", "k_proj", "v_proj": These are projections for the query, key, and value in the attention mechanism.
#"o_proj": Output projection in the attention mechanism.
#"gate_proj", "up_proj", "down_proj": Additional components for gating or other custom architectures.
#"lm_head": Language model head, used in generative tasks.

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 3407872 || all params: 7245139968 || trainable%: 0.04703666202518836


/Users/dkim1998/Documents/virtual_environment/virtual_environment/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [15]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_feature

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:

##Go here to fix error: https://github.com/huggingface/transformers/issues/26969
from datetime import datetime


project = "danny_v1-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2, #Batch Size and Gradient Accumulation:
        gradient_accumulation_steps=4, #Batch Size and Gradient Accumulation: if you decrease this it may speed up the test but use more memory
        max_steps=1000,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        bf16=False,
        optim="paged_adamw_8bit",
        gradient_checkpointing=True, #this fixes the issue
        gradient_checkpointing_kwargs={'use_reentrant':False}, #this fixes the issue
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to=None,           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

#for the sake of time, what if I lower the max steps to 100, and see how the performance gets better each time I increase it?

In [17]:
##Part 2
#Test this out on 01/09/2024

from datetime import datetime


project = "danny_v1-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=25, #original was 5, it seemed that adjusting it may have made my fan louder
        per_device_train_batch_size=1,  # Adjusted for CPU
        gradient_accumulation_steps=2,  # Adjusted for CPU
        max_steps=100, #this will change the learning speed, the lower it is the faster the speed but the higher chance it won't do well.
        learning_rate=2.5e-5, #maybe increase that?
        logging_steps=50,
        optim="adamw_torch",  # Changed to standard optimizer
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=50,
        evaluation_strategy="steps",
        gradient_checkpointing=True, #this fixes the issue
        gradient_checkpointing_kwargs={'use_reentrant':False}, #this fixes the issue
        eval_steps=50,
        do_eval=True,
        report_to=None,
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()
#for the sake of time, what if I lower the max steps to 100, and see how the performance gets better each time I increase it?
#Lets do max step 100
#Then lets do max step 300
#Max step 1000 was the one that takes forever, could be several hours to more than a day
#I may need to test try this on my work computer to use cuda (macbook was 221 minutes)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 20:13:11] [setup] RAM Tracking...
[codecarbon INFO @ 20:13:11] [setup] GPU Tracking...
[codecarbon INFO @ 20:13:11] No GPU found.
[codecarbon INFO @ 20:13:11] [setup] CPU Tracking...
[codecarbon WARNING @ 20:13:11] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 20:13:11] We saw that you have a 

{'loss': 0.8454, 'learning_rate': 1.6666666666666667e-05, 'epoch': 0.08}


[codecarbon INFO @ 20:25:15] Energy consumed for RAM : 0.004801 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:25:15] Energy consumed for all CPUs : 0.008502 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:15] 0.013303 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:25:30] Energy consumed for RAM : 0.004901 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:25:30] Energy consumed for all CPUs : 0.008679 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:30] 0.013580 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:25:45] Energy consumed for RAM : 0.005001 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:25:45] Energy consumed for all CPUs : 0.008856 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:25:45] 0.013857 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:26:00] Energy consumed for RAM : 0.005101 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:26:00] Energy consumed for all CPUs : 0.009033 kWh. Total CPU Power : 42.5 W
[co

In [ ]:
from torch.utils.checkpoint import checkpoint

In [31]:
#Part 3
#this seem to work? though it took around 2 hours
from torch.utils.checkpoint import checkpoint

trainer = transformers.Trainer(
    model=model, 
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3, 
        fp16=False,
        logging_steps=1, 
        output_dir='outputs',
        optim = "adamw_hf",
        gradient_checkpointing=True, #this fixes the issue
        gradient_checkpointing_kwargs={'use_reentrant':False} #this fixes the issue
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

[codecarbon INFO @ 23:10:37] Energy consumed for RAM : 0.055310 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:10:37] Energy consumed for all CPUs : 0.097955 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:10:37] 0.153266 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:10:38] Energy consumed for RAM : 0.052611 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:10:38] Energy consumed for all CPUs : 0.093174 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:10:38] 0.145784 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:10:38] Energy consumed for RAM : 0.053711 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:10:38] Energy consumed for all CPUs : 0.095123 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:10:38] 0.148833 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:10:39] Energy consumed for RAM : 0.042210 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:10:39] Energy consumed for all CPUs : 0.074754 kWh. Total CPU Power : 42.5 W
[co

{'loss': 0.0, 'learning_rate': 0.00046, 'epoch': 0.61}


[codecarbon INFO @ 23:12:07] Energy consumed for RAM : 0.044110 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:12:07] Energy consumed for all CPUs : 0.078119 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:12:07] 0.122229 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:12:07] Energy consumed for RAM : 0.055910 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:12:07] Energy consumed for all CPUs : 0.099018 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:12:07] 0.154928 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:12:08] Energy consumed for RAM : 0.053211 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:12:08] Energy consumed for all CPUs : 0.094237 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:12:08] 0.147447 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:12:08] Energy consumed for RAM : 0.054311 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:12:08] Energy consumed for all CPUs : 0.096185 kWh. Total CPU Power : 42.5 W
[co

{'loss': 0.0, 'learning_rate': 0.00047, 'epoch': 0.62}


[codecarbon INFO @ 23:14:29] Energy consumed for RAM : 0.044710 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:14:29] Energy consumed for all CPUs : 0.079181 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:14:29] 0.123891 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:14:33] Energy consumed for RAM : 0.044910 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:14:33] Energy consumed for all CPUs : 0.079536 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:14:33] 0.124446 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:14:34] Energy consumed for RAM : 0.048713 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:14:34] Energy consumed for all CPUs : 0.086268 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:14:34] 0.134981 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:14:36] Energy consumed for RAM : 0.046510 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:14:36] Energy consumed for all CPUs : 0.082369 kWh. Total CPU Power : 42.5 W
[co

KeyboardInterrupt: 

[codecarbon INFO @ 23:16:18] Energy consumed for RAM : 0.045610 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:16:18] Energy consumed for all CPUs : 0.080775 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:16:18] 0.126386 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:16:19] Energy consumed for RAM : 0.049413 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:16:19] Energy consumed for all CPUs : 0.087508 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:16:19] 0.136920 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:16:21] Energy consumed for RAM : 0.047210 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:16:21] Energy consumed for all CPUs : 0.083608 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:16:21] 0.130818 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:16:22] Energy consumed for RAM : 0.045011 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 23:16:22] Energy consumed for all CPUs : 0.079713 kWh. Total CPU Power : 42.5 W
[co

In [ ]:
from transformers import Trainer, TrainingArguments

project = "danny_v1-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    num_train_epochs=3,      # Total number of training epochs
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps=500,        # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,       # Strength of weight decay
    logging_dir='./logs',    # Directory for storing logs
    # Add other arguments as needed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    # Add any other necessary parameters like data_collator
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[codecarbon INFO @ 21:23:45] [setup] RAM Tracking...
[codecarbon INFO @ 21:23:45] [setup] GPU Tracking...
[codecarbon INFO @ 21:23:45] No GPU found.
[codecarbon INFO @ 21:23:45] [setup] CPU Tracking...
[codecarbon WARNING @ 21:23:45] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 21:23:46] Energy consumed for RAM : 0.000600 kWh. RAM Power : 24.0 W
[codecarbon WARNING @ 21:23:46] We saw that you have a Apple M2 Max but we don't know it. Pl

ValueError: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.

[codecarbon INFO @ 21:23:50] Energy consumed for RAM : 0.000800 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 21:23:50] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:23:50] 0.002218 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:23:50] Energy consumed for RAM : 0.012601 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 21:23:50] Energy consumed for all CPUs : 0.022318 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:23:50] 0.034919 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:23:50] Energy consumed for RAM : 0.009901 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 21:23:50] Energy consumed for all CPUs : 0.017535 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:23:50] 0.027436 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:23:50] Energy consumed for RAM : 0.011001 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 21:23:50] Energy consumed for all CPUs : 0.019484 kWh. Total CPU Power : 42.5 W
[co

In [ ]:
import torch
print(torch.__version__)


2.3.0.dev20231222


[codecarbon INFO @ 20:34:22] Energy consumed for RAM : 0.000400 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:34:22] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:22] 0.001108 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:34:30] Energy consumed for RAM : 0.001700 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:34:30] Energy consumed for all CPUs : 0.003011 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:30] 0.004711 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:34:37] Energy consumed for RAM : 0.000500 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:34:37] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:34:37] 0.001386 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:34:45] Energy consumed for RAM : 0.001800 kWh. RAM Power : 24.0 W
[codecarbon INFO @ 20:34:45] Energy consumed for all CPUs : 0.003188 kWh. Total CPU Power : 42.5 W
[co

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))
    

In [ ]:
### 01/09/2024 improtant notes about the model
#Mistral 7B, like other large language models, has an inherent limitation in terms of the number of tokens it can process in a single pass, often determined by its context window size. While the exact token limit for Mistral 7B might not be publicly documented, models of similar scale often have a maximum context window in the range of a few hundred to a few thousand tokens.

#For processing 1 million rows of data, a practical approach would be to batch the data into smaller subsets that fit within the model's token limit. You can process each batch sequentially and then aggregate the results. This method requires careful handling to ensure context and dependencies are maintained across batches. After processing, you can concatenate or summarize the outputs to achieve a comprehensive result. Keep in mind that this approach increases computational load and complexity.


##Example code where I can batch the data before when its being processed, once I'm done finetuning I need to modify it with the rest of the code
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

def process_batch(data, model, tokenizer):
    inputs = tokenizer(data, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Assume `data_rows` is a list of strings (each row of your data)
data_rows = ["row 1 data", "row 2 data", ... ]  # replace with your actual data

# Define batch size
batch_size = 1000

# Process data in batches
results = []
for i in range(0, len(data_rows), batch_size):
    batch = data_rows[i:i+batch_size]
    batch_results = process_batch(batch, model, tokenizer)
    results.extend(batch_results)

# Now, results contain the processed output for each batch



In [ ]:
##Uses mistral 7B model

tokenizer = AutoTokenizer.from_pretrained("/Users/dkim1998/Documents/Mistral_7b_v2") #Note it seems using 13B use so much memory
model = AutoModelForCausalLM.from_pretrained("/Users/dkim1998/Documents/Mistral_7b_v2")

# Encode some input text
inputs = tokenizer("who has the strongest military in the world in 2023", return_tensors="pt")

# Generate a response
outputs = model.generate(**inputs, max_length=50) #uses up to 30 gigabyte of ram just to run it
generated_text = tokenizer.decode(outputs[0])
print(generated_text)